# Распознавание Симпсонов

## Введение

### Коротко о применении нейросетей

Нейронные сети используются для решения сложных задач, которые требуют аналитических вычислений подобных тем, что делает человеческий мозг. Самыми распространенными применениями нейронных сетей является:

Классификация — распределение данных по параметрам. Например, на вход дается набор людей и нужно решить, кому из них давать кредит, а кому нет. Эту работу может сделать нейронная сеть, анализируя такую информацию как: возраст, платежеспособность, кредитная история и тд.

Предсказание — возможность предсказывать следующий шаг. Например, рост или падение акций, основываясь на ситуации на фондовом рынке.

Распознавание — в настоящее время, самое широкое применение нейронных сетей. Используется в Google, когда вы ищете фото или в камерах телефонов, когда оно определяет положение вашего лица и выделяет его и многое другое.

### Описание проекта

В рамках практической работы по дисциплине "Социально-этические вопросы ИТ" наша команда решила создать нейросеть, целью которой будет распознавание персонажей из сериала "Симпсоны" (The Simpsons).

Для обучения модели мы будем использовать датасет из 990 картинок, сгруппированных по папкам таким образом, что картинки с одним персонажем лежат в папке, название которой совпадает с именем этого персонажа.


 ![alt text](https://cdn1.thr.com/sites/default/files/imagecache/768x433/2018/09/simpsons_header-h_2018.jpg)




### Краткое описание проекта


* Применяем набор аугментаций со случайным порядком и кастомизируемыми вероятностями для каждой из трансформаций

* Маловероятно произвести надёжную валидацию при таком дисбалансе классов и малом количестве образцов, поэтому используем полностью все размеченные образцы для обучения модели. Это должно хоть немного повысить работоспособность классификации, в особенности для маленьких классов

* Для сравнения степени влияния аугментации на целевые метрики создадим 2 датасета (один с аугментацией, другой - без)

* Создадим нейронную сеть с большими дропаутами

* Добавляем случайный шум к загруженным изображениям из train

* Применяем weighted random sampler внутри data loader, чтобы бороться с дисбалансом классов

* Размер батча должен быть кратным количеству классов (то есть: 42; 84; 126; 168 и т.п.)

* Изменяем *batch size* и *learning rate* каждые 10 эпох

* Периодически сохраняем промежуточные состояния весов модели (могут пригодиться, чтобы откатить обучение)

*********************


# Импортируем библиотеки



In [0]:
import pandas as pd
import torch
import PIL
import pickle
import numpy as np
from skimage import io
import random

from tqdm import tqdm, tqdm_notebook
from PIL import Image
from pathlib import Path

from torchvision import transforms
from multiprocessing.pool import ThreadPool
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

from matplotlib import colors, pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings(action='ignore', category=DeprecationWarning)

# Фиксируем параметры генераторов случайных чисел для воспроизводимости результатов



In [0]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

DEVICE = torch.device("cuda")


In [3]:
from google.colab import drive
drive.mount('/content/gdrive/')

!unzip /content/gdrive/My\ Drive/Simpsons/simpsons4.zip

Streaming output truncated to the last 5000 lines.
  inflating: train/simpsons_dataset/moe_szyslak/pic_1026.jpg  
  inflating: train/simpsons_dataset/moe_szyslak/pic_1027.jpg  
  inflating: train/simpsons_dataset/moe_szyslak/pic_1028.jpg  
  inflating: train/simpsons_dataset/moe_szyslak/pic_1029.jpg  
  inflating: train/simpsons_dataset/moe_szyslak/pic_1030.jpg  
  inflating: train/simpsons_dataset/moe_szyslak/pic_1031.jpg  
  inflating: train/simpsons_dataset/moe_szyslak/pic_1032.jpg  
  inflating: train/simpsons_dataset/moe_szyslak/pic_1033.jpg  
  inflating: train/simpsons_dataset/moe_szyslak/pic_1034.jpg  
  inflating: train/simpsons_dataset/moe_szyslak/pic_1035.jpg  
  inflating: train/simpsons_dataset/moe_szyslak/pic_1036.jpg  
  inflating: train/simpsons_dataset/moe_szyslak/pic_1037.jpg  
  inflating: train/simpsons_dataset/moe_szyslak/pic_1038.jpg  
  inflating: train/simpsons_dataset/moe_szyslak/pic_1039.jpg  
  inflating: train/simpsons_dataset/moe_szyslak/pic_1040.jpg  
  in

# Задаём набор аугментаций со случайным порядком и вероятностями для каждой из трансформаций



In [0]:
custom_augmentations_01 = transforms.RandomOrder([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomApply([transforms.RandomRotation(degrees=10)], p=0.25),
    transforms.RandomApply([transforms.RandomResizedCrop(224, scale=(0.8, 1.25), ratio=(0.8, 1.25))], p=0.25),
    transforms.RandomApply([transforms.RandomAffine((-10,10), (0.1,0.1))], p=0.25),
    transforms.RandomPerspective(distortion_scale=0.1, p=0.25),
    transforms.RandomApply([transforms.ColorJitter(brightness=0.02,contrast=0.02,saturation=0.02,hue=(-0.02, 0.02))], p=0.01)
    ])

# Создаём класс для датасета



In [0]:
class SimpsonsDataset(Dataset):
  def __init__(self, files, mode, augmentations):
    super().__init__()
    self.files = files
    self.mode = mode
    self.augmentations = augmentations

    self.len_ = len(self.files)
    self.label_encoder = LabelEncoder()

    if self.mode != 'test':
      self.labels = [path.parent.name for path in self.files]
      self.label_encoder.fit(self.labels)

      with open('label_encoder.pkl', 'wb') as le_dump:
        pickle.dump(self.label_encoder, le_dump)

  def __len__(self):
    return self.len_

  def load_sample(self, file):
    image = Image.open(file)
    image.load()
    return image

  def __getitem__(self, index):
    transform = transforms.Compose([
      transforms.ToTensor(),
      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])                                
    ])

    x = self.load_sample(self.files[index])
    
    if self.mode=='aug':
            x = self.augmentations(x)
        
    x = self._prepare_sample(x)
    x = np.array(x / 255, dtype='float32')

    x = transform(x)
  
    if self.mode == 'test':
      return x
    else:
        
      label = self.labels[index]
      label_id = self.label_encoder.transform([label])
      y = label_id.item()
      return x, y

  def _prepare_sample(self, image):
    image = image.resize((224, 224))
    
    return np.array(image)

# Определяем пути к папкам



In [0]:
# Раскомментировать несколько нижеследующих строк, если запуск из google colab и/или в случае, если используется доступ из google drive

TRAIN_DIR = Path('/content/train/simpsons_dataset')
TEST_DIR = Path('/content/testset/testset')

#TRAIN_DIR = Path('/input/simpsons4/train')
#TEST_DIR = Path('/input/simpsons4/testset/testset')

train_files = sorted(list(TRAIN_DIR.rglob('*.jpg')))
test_files = sorted(list(TEST_DIR.rglob('*.jpg')))

# Использем все 100% доступных маркированных образцов для обучения модели 




Вряд ли возможно провести надежную валидацию с учетом такого классового дисбаланса и разреженности образцов. Поэтому используем полностью все размеченные образцы для обучения модели.

Это должно повысить надежность модели, особенно для небольших классов

для сравнения степени влияния аугментации на целевые метрики создадим 2 датасета (один с аугментацией (**aug**), другой - без ()**no_aug**)



In [0]:
from sklearn.utils import shuffle

train_labels = [path.parent.name for path in train_files]

aug_files = shuffle(train_files, random_state = 0)
no_aug_files = shuffle(train_files, random_state = 0)

aug_dataset = SimpsonsDataset(train_files, mode='aug', augmentations=custom_augmentations_01)
no_aug_dataset = SimpsonsDataset(no_aug_files, mode='no_aug', augmentations=None)

# Создадим нейронную сеть с большими дропаутами


In [0]:
class SimpleModel(nn.Module):
  
    def __init__(self, n_classes):
        super().__init__()

        self.c1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=1, padding=2),
            nn.Dropout2d(0.8),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)            
        )
        self.c2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2),
            nn.Dropout2d(0.8),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.c3 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=2),
            nn.Dropout2d(0.8),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.c4 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=96, kernel_size=5, stride=1, padding=2),
            nn.Dropout2d(0.8),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.c5 = nn.Sequential(
            nn.Conv2d(in_channels=96, out_channels=160, kernel_size=5, stride=1, padding=2),
            nn.Dropout2d(0.8),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.f1 = nn.Sequential(
            nn.Linear(7 * 7 * 160, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
        )
        self.f2 = nn.Sequential(
            nn.Linear(1024, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU()
        )        
        self.f3 = nn.Linear(1024, n_classes)

    def forward(self, x):
        x = self.c1(x)
        x = self.c2(x)
        x = self.c3(x)
        x = self.c4(x)
        x = self.c5(x)
        x = x.view(-1, 7 * 7 * 160)
        x = self.f1(x)
        x = self.f2(x)

        logits = self.f3(x)
        return logits

# Определяем функции: train(), fit_epoch(), eval_epoch()


* Добавляем случайный шум к загруженным изображениям из *train*

* применяем weighted random sampler внутри data loader, чтобы бороться с дисбалансом классов
* изменяем *batch size* и *learning rate* каждые 10 эпох

          


 ![alt text](https://cnet3.cbsistatic.com/img/C_76GS_9sar6cHjcZtrq8Eo1xFE=/1092x0/2015/05/06/9e6093ac-cac7-46d8-b502-2c52faffc2fb/thesimpsons.jpg)






In [0]:
def fit_aug_epoch(model, aug_loader, criterion, optimizer):
    running_loss = 0.0
    running_corrects = 0
    processed_data = 0
  
    for inputs, labels in aug_loader:
        # добавление случайного шума
        inputs = inputs + torch.empty(*inputs.size()).normal_(0, 0.0001)
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        preds = torch.argmax(outputs, 1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        processed_data += inputs.size(0)
              
    aug_loss = running_loss / processed_data
    aug_acc = running_corrects.cpu().numpy() / processed_data
    return aug_loss, aug_acc

 
def eval_no_aug_epoch(model, no_aug_loader, criterion):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    processed_size = 0

    for inputs, labels in no_aug_loader:
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)

        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            preds = torch.argmax(outputs, 1)

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        processed_size += inputs.size(0)
    no_aug_loss = running_loss / processed_size
    no_aug_acc = running_corrects.double() / processed_size
    return no_aug_loss, no_aug_acc
  
def train(aug_files, no_aug_files, model, epochs, batch_size, learning_rate):
    
    y_aug = pd.Series(aug_dataset.labels)
    
    # считаем образцы в каждом классе
    class_sample_count = y_aug.value_counts().to_dict()
    samples_counts = np.array(list(map(class_sample_count.get, y_aug)))
    
    # рассчитываем веса для каждого класса
    samples_weight = 1./samples_counts
    samples_weight = torch.from_numpy(samples_weight)
    
    # создаём weighted random sampler и применяем его в train_loader
    sampler = torch.utils.data.WeightedRandomSampler(samples_weight.type('torch.DoubleTensor'), num_samples=len(y_aug))
    aug_loader = DataLoader(aug_dataset, batch_size=batch_size, shuffle=False, sampler=sampler)

    no_aug_loader = DataLoader(no_aug_dataset, batch_size=batch_size, shuffle=False)

    history = []
    log_template = "\nEpoch {ep:03d} aug_loss: {t_loss:0.4f} \
    no_aug_loss {v_loss:0.4f} aug_acc {t_acc:0.4f} no_aug_acc {v_acc:0.4f}"

    with tqdm(desc="epoch", total=epochs) as pbar_outer:
      
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        #scheduler = torch.optim.lr_scheduler.StepLR(opt, step_size = 5, gamma = 0.5)
        criterion = nn.CrossEntropyLoss()

        for epoch in range(epochs):
            aug_loss, aug_acc = fit_aug_epoch(model, aug_loader, criterion, optimizer)
            print("loss", aug_loss)
            
            no_aug_loss, no_aug_acc = eval_no_aug_epoch(model, no_aug_loader, criterion)
            history.append((aug_loss, aug_acc, no_aug_loss, no_aug_acc))
            
            pbar_outer.update(1)
            tqdm.write(log_template.format(ep=epoch+1, t_loss=aug_loss,\
                                           v_loss=no_aug_loss, t_acc=aug_acc, v_acc=no_aug_acc))
            #scheduler.step()
    return history

# Определяем функцию *predict()*


In [0]:
def predict(model, aug_loader):
    with torch.no_grad():
        logits = []
    
        for inputs in test_loader:
            inputs = inputs.to(DEVICE)
            model.eval()
            outputs = model(inputs).cpu()
            logits.append(outputs)
            
    probs = nn.functional.softmax(torch.cat(logits), dim=-1).numpy()
    return probs

# Начнём обучать модель и будем периодически сохранять веса модели: 

размер батча должен быть кратным количеству классов (то есть: 42; 84; 126; 168 и т.п.)

# Первые 10 эпох
> epochs=10, batch_size=168, learning_rate=0.01

In [0]:

n_classes = len(np.unique(train_labels))
model = SimpleModel(n_classes).to(DEVICE)

# начнём обучение с 10 эпох: размер батча должен быть кратным количеству классов (то есть: 1*42=42; 2*42=84; 3*42=126; 4*42=168 и т.п.)
history = train(aug_dataset, no_aug_dataset, model=model, epochs=10, batch_size=168, learning_rate=0.01)

# сохраним веса модели после 10 эпох
torch.save(model.state_dict(), "Simple_Simpson_model_001_weights_01.pth")

# сделаем визуализацию
loss, acc, val_loss, val_acc = zip(*history)
plt.figure(figsize=(15, 9))
plt.plot(loss, label="aug_loss")
plt.plot(val_loss, label="no_aug_loss")
plt.legend(loc='best')
plt.xlabel("epochs")
plt.ylabel("loss")
plt.show()

epoch:   0%|          | 0/10 [00:00<?, ?it/s]

loss 4.097498580466737


epoch:  10%|█         | 1/10 [08:20<1:15:00, 500.06s/it]


Epoch 001 aug_loss: 4.0975     no_aug_loss 3.7258 aug_acc 0.0247 no_aug_acc 0.0262
loss 3.65059496497297


# 20 эпох
> epochs=10, batch_size=126, learning_rate=0.01

In [0]:
# продолжим обучение ещё 10 эпох: уменьшим размер батча и learning rate
history = train(aug_dataset, no_aug_dataset, model=model, epochs=10, batch_size=126, learning_rate=0.01)

# сохраним веса модели после 20 эпох
torch.save(model.state_dict(), "Simple_Simpson_model_001_weights_02.pth")

# делаем визуализацию
loss, acc, val_loss, val_acc = zip(*history)
plt.figure(figsize=(15, 9))
plt.plot(loss, label="aug_loss")
plt.plot(val_loss, label="no_aug_loss")
plt.legend(loc='best')
plt.xlabel("epochs")
plt.ylabel("loss")
plt.show()

# 30 эпох
> epochs=10, batch_size=84, learning_rate=0.001

In [0]:

history = train(aug_dataset, no_aug_dataset, model=model, epochs=10, batch_size=84, learning_rate=0.001)

torch.save(model.state_dict(), "Simple_Simpson_model_001_weights_03.pth")

# делаем визуализацию
loss, acc, val_loss, val_acc = zip(*history)
plt.figure(figsize=(15, 9))
plt.plot(loss, label="aug_loss")
plt.plot(val_loss, label="no_aug_loss")
plt.legend(loc='best')
plt.xlabel("epochs")
plt.ylabel("loss")
plt.show()

# 40 эпох 
> epochs=10, batch_size=42, learning_rate=0.001

In [0]:

history = train(aug_dataset, no_aug_dataset, model=model, epochs=10, batch_size=42, learning_rate=0.001)

torch.save(model.state_dict(), "Simple_Simpson_model_001_weights_04.pth")

# делаем визуализацию
loss, acc, val_loss, val_acc = zip(*history)
plt.figure(figsize=(15, 9))
plt.plot(loss, label="aug_loss")
plt.plot(val_loss, label="no_aug_loss")
plt.legend(loc='best')
plt.xlabel("epochs")
plt.ylabel("loss")
plt.show()

# 50 эпох 
> epochs=10, batch_size=42, learning_rate=0.001

In [0]:

history = train(aug_dataset, no_aug_dataset, model=model, epochs=10, batch_size=42, learning_rate=0.001)

torch.save(model.state_dict(), "Simple_Simpson_model_001_weights_05.pth")

# делаем визуализацию
loss, acc, val_loss, val_acc = zip(*history)
plt.figure(figsize=(15, 9))
plt.plot(loss, label="aug_loss")
plt.plot(val_loss, label="no_aug_loss")
plt.legend(loc='best')
plt.xlabel("epochs")
plt.ylabel("loss")
plt.show()

# 60 эпох
> epochs=10, batch_size=42, learning_rate=0.001

In [0]:

history = train(aug_dataset, no_aug_dataset, model=model, epochs=10, batch_size=42, learning_rate=0.0001)

torch.save(model.state_dict(), "Simple_Simpson_model_001_weights_06.pth")

# делаем визуализацию
loss, acc, val_loss, val_acc = zip(*history)
plt.figure(figsize=(15, 9))
plt.plot(loss, label="aug_loss")
plt.plot(val_loss, label="no_aug_loss")
plt.legend(loc='best')
plt.xlabel("epochs")
plt.ylabel("loss")
plt.show()

# Делаем предсказание на тестовом датасете и генерируем submission

In [0]:
label_encoder = pickle.load(open("label_encoder.pkl", 'rb'))

test_dataset = SimpsonsDataset(test_files, mode="test", augmentations=None)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=42)
probs = predict(model, test_loader)

preds = label_encoder.inverse_transform(np.argmax(probs, axis=1))
test_filenames = [path.name for path in test_dataset.files]

submit = pd.DataFrame({'Id': test_filenames, 'Expected': preds})
submit.head()

submit.to_csv('submission.csv', index=False)



 ![alt text](https://i.guim.co.uk/img/media/88f6b98714035656cb18fb282507b60e82edb0d7/0_35_2560_1536/master/2560.jpg?width=1920&quality=85&auto=format&fit=max&s=f09a5ef4ac4c0e214594f65315390016)



                            
